# Qdrant Indexing

In [ ]:
from datasets import load_dataset


dataset = load_dataset('BeIR/scifact', 'corpus', split='corpus')
dataset[0]

# Dense Embedding

`Trong phần này, chúng ta sẽ chọn thư viện FastEmbedded với các mô hình embedding được huấn luyện từ trước.
Do sử dụng ONNX, nên các mô hình này có thể chạy một cách hiệu quả ngay cả trên CPU. Mô hình MiniLM-L6-v2 là mô hình nhẹ và phù hợp để bắt đầu làm quen với Qdrant`

In [10]:
from fastembed.embedding import TextEmbedding

dense_embedding_model = TextEmbedding('sentence-transformers/all-MiniLM-L6-v2')
dense_embeddings = list(dense_embedding_model.passage_embed(dataset['text'][0:1]))
print(len(dense_embeddings))


Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 1670.90it/s]


1


# Sparse Embedding


In [11]:
from fastembed.sparse.bm25 import Bm25

bm25_embedding_model = Bm25('Qdrant/bm25')
bm25_embeddings = list(bm25_embedding_model.passage_embed(dataset['text'][0:1]))
print(len(bm25_embeddings))

Fetching 29 files: 100%|██████████| 29/29 [00:00<00:00, 29071.42it/s]


1


# Late Interactionn Embeddings

`Một cách tiếp cận mới là mô hình Late Interaction hay gọi là tương tác muộn, thay vì embedding các token và sau đó tổng hợp các vector embedding của các token để tạo thành vector embedding của một câu thì cơ chế tương tác muộn sẽ giữ lại các vector embedding của các token và sau đó tính toán độ tương đồng giữa các token của câu query và token của tài liệu. Tham số đánh giá được sử dụng trong cơ chế này là MAXSIM. Tham số này sẽ được sử dụng để đánh giá độ tương đồng giữa câu truy vấn và tài liệu `

`Một mô hình sử dụng cơ chế tương tác muộn này là "ColBert"`

In [12]:
from fastembed.late_interaction import LateInteractionTextEmbedding

late_interaction_embedding_model = LateInteractionTextEmbedding('colbert-ir/colbertv2.0')
late_interaction_embeddings = list(late_interaction_embedding_model.passage_embed(dataset['text'][0:1]))
print(len(late_interaction_embeddings))

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 1110.13it/s]


1


# Putting Data to Qdrant Collection

`Tất cả các vector hiện có được đưa vào Qdrant Collectionn. Việc giữ các vector này trong một bộ duy nhất cho phép kết hợp các loại embedding khác nhau và thậm chí tạo ra một pipeline phức tạp nhiều bước.`

`Để thực hiện chạy Qdrant, mở terminal và gõ lệnh sau đây: docker run -d -p 6333:6333 -p 6334:6334 qdrant/qdrant:v1.10.0`


In [13]:
from qdrant_client import QdrantClient
from qdrant_client.http import models

client = QdrantClient(
    url='https://618dc965-c6d2-4c3b-93bd-26050204f910.us-east4-0.gcp.cloud.qdrant.io:6333',
    api_key= 'AVW6PnZn-3ld6bw33exaLaohJd8JymLDne4Mk4nIE298dhtB_XS5hQ'
)
client.create_collection(
    "scifact1",
    vectors_config={
        "all-MiniLM-L6-v2": models.VectorParams(
            size=len(dense_embeddings[0]),
            distance=models.Distance.COSINE,
        ),
        "colbertv2.0": models.VectorParams(
            size=len(late_interaction_embeddings[0][0]),
            distance=models.Distance.COSINE,
            multivector_config=models.MultiVectorConfig(
                comparator=models.MultiVectorComparator.MAX_SIM,
            )
        ),
    },
    sparse_vectors_config={
        "bm25": models.SparseVectorParams(
            modifier=models.Modifier.IDF,
        )
    }
)

True

In [ ]:
import tqdm

batch_size = 4
for batch in tqdm.tqdm(dataset.iter(batch_size=batch_size), 
                       total=len(dataset) // batch_size):
    dense_embeddings = list(dense_embedding_model.passage_embed(batch["text"]))
    bm25_embeddings = list(bm25_embedding_model.passage_embed(batch["text"]))
    late_interaction_embeddings = list(late_interaction_embedding_model.passage_embed(batch["text"]))
    
    client.upload_points(
        "scifact1",
        points=[
            models.PointStruct(
                id=int(batch["_id"][i]),
                vector={
                    "all-MiniLM-L6-v2": dense_embeddings[i].tolist(),
                    "bm25": bm25_embeddings[i].as_object(),
                    "colbertv2.0": late_interaction_embeddings[i].tolist(),
                },
                payload={
                    "_id": batch["_id"][i],
                    "title": batch["title"][i],
                    "text": batch["text"][i],
                }
            )
            for i, _ in enumerate(batch["_id"])
        ],
        # We send a lot of embeddings at once, so it's best to reduce the batch size.
        # Otherwise, we would have gigantic requests sent for each batch and we can
        # easily reach the maximum size of a single request.
        batch_size=batch_size,  
    )

In [9]:
from qdrant_client import QdrantClient, models


client = QdrantClient(
    url='https://618dc965-c6d2-4c3b-93bd-26050204f910.us-east4-0.gcp.cloud.qdrant.io:6333',
    api_key= 'AVW6PnZn-3ld6bw33exaLaohJd8JymLDne4Mk4nIE298dhtB_XS5hQ'
)

In [10]:
client.count('scifact3')

CountResult(count=5183)

In [1]:
from qdrant_client import QdrantClient, models
from fastembed.embedding import TextEmbedding
from fastembed.sparse.bm25 import Bm25
from fastembed.late_interaction import LateInteractionTextEmbedding
from config import load_config



CONFIG = load_config()

dense_embedding_model = TextEmbedding(CONFIG['qdrant']['dense_embedding'])
sparse_embedding_model = Bm25(CONFIG['qdrant']['sparse_embedding'])
late_interaction_embedding_model = LateInteractionTextEmbedding(CONFIG['qdrant']['late_interaction_embedding'])



class HybridSearch:
    def __init__(self):
        self.client = QdrantClient(
            url = CONFIG['qdrant']['url'],
            api_key=CONFIG['qdrant']['api_key']
        )
    def query_docs(self, query_text):
        dense_embedding_query = next(dense_embedding_model.query_embed(query_text))
        sparse_embedding_query = next(sparse_embedding_model.query_embed(query_text))
        late_interaction_embedding_query = next(late_interaction_embedding_model.query_embed(query_text))
        prefetch = [
            models.Prefetch(
                query=dense_embedding_query,
                using = 'all-MiniLM-L6-v2',
                limit = 20
            ),
            models.Prefetch(
                query=models.SparseVector(**sparse_embedding_query.as_object()),
                using = 'bm25',
                limit = 20
            ),
            models.Prefetch(
                query = late_interaction_embedding_query,
                using = 'colbertv2.0',
                limit = 20
            )
        ]
        print('DONE')
        responses = self.client.query_points(
            collection_name=CONFIG['qdrant']['collection_name'],
            prefetch=prefetch,
            query = models.FusionQuery(
                fusion=models.Fusion.RRF
            ),
            with_payload=True,
            limit = 10
        )
        score_threshold = CONFIG['qdrant']['thresh_score']
        docs = ''
        for i, response in enumerate(responses.points):
            if response.score >= score_threshold:
                docs += f'Example{i+1}:\nTitle:{response.payload['title']}\nParagraph: {response.payload['text']}\n'
        return docs

c:\Users\Admin\anaconda3\envs\RAG_Chatbot_3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-04 22:19:28.299 | WARNING  | fastembed.embedding:<module>:7 - DefaultEmbedding, FlagEmbedding, JinaEmbedding are deprecated.Use from fastembed import TextEmbedding instead.
Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 5018.31it/s]


In [ ]:
search = HybridSearch()
query = "What is the impact of COVID-19 on the environment?"
docs = search.query_docs(query)